<a href="https://colab.research.google.com/github/furlancad/Estudo-PMF/blob/main/Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beta

Beta é a medida que exprime a resposta do preço do ativo em relação ao risco sistemático ou risco de mercado.

Amplamente utilizado no modelo CAPM - Capital Asset Pricing Model como proxy de volatilidade, o Beta baseia-se na linearidade entre os dados de mercado e o dado ativo, buscando capturar a inflûencia do primeiro no segundo.

In [1]:
!pip install yfinance -q
!pip install plotly -q

# Regressão Linear

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from tempfile import template

# 1° Capturar os dados

In [3]:
tickers = ['^BVSP', 'PETR4.SA']
inicio = "2019-09-21"
fim = "2022-09-21"

ativos = pd.DataFrame()

for i in tickers:
  df = yf.download(i, start=inicio, end=fim)['Adj Close']
  df.rename(i, inplace=True)
  ativos = pd.concat([ativos, df], axis=1)
  ativos.index.name='Date'

ativos

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,^BVSP,PETR4.SA
Date,,
2019-09-23,104638.0,9.909776
2019-09-24,103876.0,9.834045
2019-09-25,104481.0,9.859288
2019-09-26,105319.0,9.989112
2019-09-27,105078.0,9.974687
...,...,...
2022-09-14,110547.0,21.098570
2022-09-15,109954.0,21.057892
2022-09-16,109280.0,20.868059


# 2° Calcular os retornos e dropar os NaNs

In [4]:
retornos = ativos.pct_change()
retornos.dropna(inplace=True)

In [5]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=retornos['^BVSP'], y=retornos['PETR4.SA'], name='PETR4', mode='markers'))

fig.update_layout(title='', template = 'simple_white', paper_bgcolor='#f7f8fa', margin=dict(l=20, r=20, t=20, b=20), width=600, height=500)

fig.show()

# 3° Calcular a regressão e extrair o Beta

In [6]:
Y = retornos['PETR4.SA']
X = retornos['^BVSP']

X = sm.add_constant(X)

modelo = sm.OLS(Y,X)
resultado = modelo.fit()
beta = resultado.params[1]

In [7]:
beta

1.3308340907190717



*   Beta > 1 ação mais volátil que o mercado
*   Beta < 0 ação menos volátil que o mercado



# Rolling Beta

In [8]:
from statsmodels.regression.rolling import RollingOLS

In [9]:
tickers = ['^BVSP', 'PETR4.SA']
inicio = "2010-09-21"
fim = "2022-09-21"

ativos = pd.DataFrame()

for i in tickers:
  df = yf.download(i, start=inicio, end=fim)['Adj Close']
  df.rename(i, inplace=True)
  ativos = pd.concat([ativos, df], axis=1)
  ativos.index.name='Date'

ativos

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,^BVSP,PETR4.SA
Date,,
2010-09-21,67719.0,7.683409
2010-09-22,68325.0,7.575519
2010-09-23,68794.0,7.814623
2010-09-24,68196.0,7.668825
2010-09-27,68816.0,7.727145
...,...,...
2017-11-02,NaN,5.770576
2017-11-15,NaN,5.241321
2017-11-20,NaN,5.470096


In [10]:
retornos = ativos.pct_change()
retornos.dropna(inplace=True)

In [11]:
Y = retornos['PETR4.SA']
X = retornos['^BVSP']

X = sm.add_constant(X)

modelo = RollingOLS(Y,X, window=252)
resultado = modelo.fit()
params = resultado.params.copy()
params.index = np.arange(1, params.shape[0] + 1)

In [12]:
beta = pd.DataFrame(pd.to_numeric(params['^BVSP']))
beta['^BVSP'] = beta['^BVSP'].astype(float)

In [14]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=retornos.index, y=beta['^BVSP'], name='PETR4'))

fig.update_layout(template = 'simple_white', width=900, height=500)

fig.show()